# Enhancing RAG with Neo4j Knowledge Graph


Import modules and instantiate connections and models

In [15]:
import snowflake.connector
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate


import config
from llm import Cortex
from ner import EntityFinder
from graph import NewsGraphClient


snowflake_connection = snowflake.connector.connect(**config.SNOWFLAKE_CONNECTION_PARAMS)
model = Cortex(connection=snowflake_connection, model=config.CHAT_MODEL)
entity_finder = EntityFinder(config.RELEVANT_LABELS)
db = NewsGraphClient()

# Define helper functions
def map_candidates_to_context(candidates: list[dict[str, str]]) -> str:
    context_str = ', '.join(
        f"(:{c['label']} {{ name: '{c['name']}' }}"
        for c in candidates
    )
    return context_str


def map_records_to_context(db_records: list[dict]) -> str:
    context_str = ('='*5+'\n').join(
        '/n'.join(f"{k}: {v}" for k, v in record.items())
        for record in db_records
    )
    return context_str


/home/djordje/.virtualenvs/bbuzz24/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
/home/djordje/.virtualenvs/bbuzz24/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [2]:
CYPHER_GENERATION_TEMPLATE = """Based on the graph schema below, write a Cypher query that answers the user's question. 
Use only the node labels, relationships and properties provided in the schema:
{schema}
Entities in the question map to the following database values:
{entities_list}

Here are some examples: 
Example 1: For the question "List 10 titles of articles mentioning Ursula von der Leyen" and the entity list "(:Person {{ name: 'Ursula von der Leyen' }}, (:Person {{ name: 'Ursula v. d. Leyn' }}" the generated Cypher query should be 
"MATCH (a:Article)-[:CONTAINS]->(c:Chunk)-[:MENTIONS]->(o:Person) WHERE o.name IN ['Ursula von der Leyen', 'Ursula v. d. Leyn'] RETURN DISTINCT a.title LIMIT 10"

Example 2: For the question "How many sources mention the EU commission?" and the entity list "(:Organization {{ name: 'EU-Kommission' }}" the generated Cypher query should be 
"MATCH (s:Source)-[:PUBLISHED]->(a:Article)-[:CONTAINS]->(c:Chunk)-[:MENTIONS]->(o:Organization) WHERE o.name IN ['EU-Kommission'] WITH DISTINCT s RETURN count(s)"

Example 3: For the question "News about France and Macron?" and the entity list "(:Location {{ name: 'France' }}, (:Person {{ name: 'Emmanuel Macron' }}" the generated Cypher query should be 
"MATCH (c:Chunk)-[:MENTIONS]->(o:Location) WHERE o.name = 'France' UNION MATCH (c:Chunk)-[:MENTIONS]->(o:Person) WHERE o.name = 'Emmanuel Macron' RETURN c.text LIMIT 10"

Question: {question}
Cypher query:"""

## Generate a Cypher query

In [28]:
question = 'How many articles are there about the SPD?'
# Get entities from text
mentioned_entities = entity_finder.find(question)
# Perform fulltext search
candidates = db.lookup_mentioned_entities(mentioned_entities)
print(candidates)
candidate_context = map_candidates_to_context(candidates)
print(candidate_context)

[{'uid': 'Organization:lfpV7XYWRaG6', 'name': 'SPD', 'label': 'Organization', 'score': 3.131917715072632}, {'uid': 'Organization:JkZanoJ8QyOj', 'name': 'SPD-Chefin', 'label': 'Organization', 'score': 2.447997808456421}, {'uid': 'Organization:7Wm84tMGSF-Z', 'name': 'SPD-Politiker', 'label': 'Organization', 'score': 2.447997808456421}]
(:Organization { name: 'SPD' }, (:Organization { name: 'SPD-Chefin' }, (:Organization { name: 'SPD-Politiker' }


In [29]:
# Define prompt
cypher_prompt = ChatPromptTemplate.from_messages([
    ("system", "Given an input question, convert it to a Cypher query. No pre-amble.",),
    ("human", CYPHER_GENERATION_TEMPLATE),
])
# Define chain
cypher_chain = cypher_prompt | model | StrOutputParser()
# Generate Cypher query with found entities
generated_query = cypher_chain.invoke({
    'question': question,
    'entities_list': candidate_context,
    'schema': db.graph.schema
})
print(generated_query)

 MATCH (a:Article)-[:CONTAINS]->(c:Chunk)-[:MENTIONS]->(o:Organization) WHERE o.name IN ['SPD', 'SPD-Chefin', 'SPD-Politiker'] WITH DISTINCT a RETURN count(a)


## Perform query and generate an answer

In [30]:

# Perform query
response = db.query(generated_query)
context = map_records_to_context(response)
print(context)

count(a): 8


In [31]:
# Define prompt and chain
ANSWER_PROMPT_TEMPLATE = (
    "Answer the question below in appropriate detail, given the following context. "
    # "Think step by step before providing a detailed answer. "
    "The context was retrieved from the database by the following query:\n\n"
    "Query: {query}\n\n"
    "Context:\n{context}\n\n"
    "Question: {question}\n\n"
    "Answer: "
)
answer_prompt = ChatPromptTemplate.from_template(ANSWER_PROMPT_TEMPLATE)
answer_chain = answer_prompt | model | StrOutputParser()

In [32]:
# Populate context and generate answer
answer = answer_chain.invoke(
    {'question': question, 'context': context, 'query': generated_query}
)
print(answer)

 There are 8 articles about the SPD.
